# Automação Web e Busca de Informações com Python

#### Desafio: 

Preços dos produtos de uma importadora são vinculado a cotação de:
- Dólar
- Euro
- Ouro

Pegar na internet, de forma automática, a cotação desses 3 itens (dólar, euro, ouro) e saber quanto devemos cobrar pelos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1o2lpxoi9heyQV1hIlsHXWSfDkBPtze-V?usp=sharing


Ferramentas:
- selenium
- webdriver
- jupyter notebook 
- python 

In [37]:
from selenium import webdriver                                    #sempre importar 
from selenium.webdriver.common.keys import Keys                   #permite controlar\escrever

#abrir navegador
navegador = webdriver.Chrome()
navegador.get('https://www.google.com/')

#Passo 1 - pegar cotação do Dolár
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('cotação dólar')

navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

cotacao_dolar = navegador.find_element_by_xpath('//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(cotacao_dolar)                                                                                                          #pega atributo do elemento


#Passo 2 - pegar cotação do Euro
navegador.get('https://www.google.com/')
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('cotação euro')

navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

cotacao_euro = navegador.find_element_by_xpath('//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(cotacao_euro) 


#Passo 3 - pegar cotação do Ouro
navegador.get('https://www.melhorcambio.com/ouro-hoje')

cotacao_ouro = navegador.find_element_by_xpath('//*[@id="comercial"]').get_attribute('value')
cotacao_ouro = cotacao_ouro.replace(',','.')
print(cotacao_ouro) 

#fechar navegador
navegador.quit()


5.208867
6.18135726
304.43


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [38]:
#Passo 4 - importar a lista de produtos
import pandas as pd

tabela = pd.read_excel('Produtos.xlsx')
display(tabela)

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Margem,Preço Final
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [39]:
#Passo 5 - recalcular o preço de cada produto
#atualizar cotação 
#tabela[linha,coluna] = valor
tabela.loc[tabela['Moeda'] == 'Dólar','Cotação'] = float(cotacao_dolar)
tabela.loc[tabela['Moeda'] == 'Euro','Cotação'] = float(cotacao_euro)
tabela.loc[tabela['Moeda'] == 'Ouro','Cotação'] = float(cotacao_ouro)


#atualizar preço base reais (preço base original * cotação)
tabela['Preço Base Reais'] = tabela['Preço Base Original'] * tabela['Cotação']

#atualizar preço final (preço base reais * margem)
tabela['Preço Final'] = tabela['Preço Base Reais'] * tabela['Margem']


display(tabela)

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Margem,Preço Final
0,Câmera Canon,999.99,Dólar,5.208867,5208.814911,1.40,7292.340876
1,Carro Renault,4500.00,Euro,6.181357,27816.107670,2.00,55632.215340
2,Notebook Dell,899.99,Dólar,5.208867,4687.928211,1.70,7969.477959
3,IPhone,799.00,Dólar,5.208867,4161.884733,1.70,7075.204046
4,Carro Fiat,3000.00,Euro,6.181357,18544.071780,1.90,35233.736382
5,Celular Xiaomi,480.48,Dólar,5.208867,2502.756416,2.00,5005.512832
6,Joia 20g,20.00,Ouro,304.430000,6088.600000,1.15,7001.890000


### Agora vamos exportar a nova base de preços atualizada

In [40]:
#Passo 6 - salvar os novos preços dos produtos em nova tabela
tabela.to_excel('Produtos_novo.xlsx',index=False)  #tira o índice da tabela 